In [1]:
import mmh3
import hashlib

In [2]:
class BloomFilter:
    def __init__(self, size, n_hash):
        self.m = size
        self.k = n_hash
        self.n = 0
        self.bloomFilter = [0]*size
    def get_indices(self, item):
        index = []
        for i in range(1, self.k+1):
            # hash functions
            index.append((hash(item) + i*mmh3.hash(item)) % self.m)
        return index
    def add(self, item):
        for i in self.get_indices(item):
            self.bloomFilter[i] = 1
        self.n += 1
    def lookup(self, key):
        for i in self.get_indices(key):
            if self.bloomFilter[i] == 0:
                return False
        return True
    def get_length(self):
        return self.n
    def get_fp_rate(self, train_data, test_data):
        tp = fp = tn = fn = 0
        for data in test_data:
            is_found = self.lookup(data)
            is_present = data in train_data
            if is_found and is_present:
                tp += 1
            elif is_found and not is_present:
                fp += 1
            elif not is_found and not is_present:
                tn += 1
            elif not is_found and is_present:
                fn += 1
            fpr = fp/(fp + tn)
        return fpr, tp, fp, tn, fn

In [3]:
bloom = BloomFilter(100, 2)
print('Size of filter: {}'.format(bloom.m))
print('Number of hash functions: {}'.format(bloom.k))

Size of filter: 100
Number of hash functions: 2


In [4]:
p1 = 'We hold these truths to be self-evident, that all men are created equal, \
that they are endowed, by their Creator, with certain unalienable rights, \
that among these are life, liberty, and the pursuit of happiness. \
That to secure these rights, governments are instituted among men, \
deriving their just powers from the consent of the governed, \
that whenever any form of government becomes destructive of these ends, \
it is the right of the people to alter or to abolish it, and to institute new government, \
laying its foundation on such principles, and organizing its powers in such form, \
as to them shall seem most likely to effect their safety and happiness.'

p2 = 'He has forbidden his governors to pass laws of immediate and pressing importance, \
unless suspended in their operations till his assent should be obtained; and when so suspended, \
he has utterly neglected to attend to them.'

txt1 = p1.split()
txt2 = p2.split()

for w in txt1:
    bloom.add(w)
print('number of items in the filter: {}'.format(bloom.n))

number of items in the filter: 110


In [5]:
print('items in the filter: {}'.format(bloom.bloomFilter))
print('Size of filter: {}'.format(bloom.m))

items in the filter: [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Size of filter: 100


In [6]:
# prepare sets for experiments
# in_p1_not_p2 = set([w for w in txt1 if w not in txt2])
in_p2_not_p1 = [w for w in txt2 if w not in txt1]
in_p1_in_p2 = set([w for w in txt1 if w in txt2])
# p1_and_p2 = set([w for w in txt1 for w in txt2])
train, test = txt1, txt2

In [7]:
# Expectation: false positive rate, false positives, true negatives
fp = tn = 0
TN = []
FP = []
N = len(in_p2_not_p1)
for w in in_p2_not_p1: 
    q = bloom.lookup(w)
    if q: 
        fp += 1
        FP.append(w)
    else: 
        tn += 1 # False means absolute absent in the filter (true negatives)
        TN.append(w)
fpr = fp/(fp+tn)  # false positive rate

# Results:
print('fpr = ', fpr)
print('fpr, tp, fp, tn, fn  = {}'.format(bloom.get_fp_rate(train, test)))
print('fp = ', fp, ', tn = ', tn, ', N = ', N)
print('TN = ', TN)
print('FP = ', FP)

fpr =  0.8148148148148148
fpr, tp, fp, tn, fn  = (0.8148148148148148, 9, 22, 5, 0)
fp =  22 , tn =  5 , N =  27
TN =  ['obtained;', 'when', 'so', 'he', 'utterly']
FP =  ['He', 'has', 'forbidden', 'his', 'governors', 'pass', 'laws', 'immediate', 'pressing', 'importance,', 'unless', 'suspended', 'operations', 'till', 'his', 'assent', 'should', 'suspended,', 'has', 'neglected', 'attend', 'them.']


In [8]:
# Expectation: should return all true positives
TP = []
tp = 0
for w in test:
    q = bloom.lookup(w)
    if q and (w in in_p1_in_p2):
        tp += 1
        TP.append(w)
print('tp = ', tp)
print('TP = ', TP)

tp =  9
TP =  ['to', 'of', 'and', 'in', 'their', 'be', 'and', 'to', 'to']
